<a href="https://colab.research.google.com/github/RMoulla/PTS/blob/main/Topic_Modeling_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling sur des avis de consommateurs

Dans ce tutoriel, nous allons utiliser une technique de topic modeling pour extraire des thématiques significatives à partir de données textuelles. Le jeu de données choisi est composé d'avis de clients sur des produits électroniques vendus sur Amazon. Ce dataset comprend des informations telles que le texte de l'avis, la note globale attribuée au produit, un résumé de l'avis, ainsi que des informations temporelles liées à chaque avis.

La technique utilisée ici est Latent Dirichlet Allocation (LDA), un modèle statistique adapté pour l'identification de thématiques latentes dans de vastes ensembles de données textuelles.

LDA est un modèle génératif probabiliste qui permet de découvrir des groupes de mots (appelés "topics" ou thématiques) dans un corpus de documents. Il est basé sur deux hypothèses principales :

* Chaque document est un mélange de sujets : LDA suppose que chaque document dans notre ensemble de données peut être décrit par un certain nombre de sujets différents. Par exemple, un avis sur un smartphone pourrait inclure des sujets tels que la performance de la batterie, la qualité de l'appareil photo, ou l'expérience utilisateur.

* Chaque sujet est un mélange de mots : Chaque sujet identifié par LDA est caractérisé par un ensemble de mots. Par exemple, le sujet "performance de la batterie" pourrait inclure des mots comme "longue durée", "recharge", "autonomie", etc.


In [ ]:
!pip install pyldavis

## Importation des librairies nécessaires

Ce code permet d'importer l'ensemble des bibliothèques nécessaires pour le script. **pandas** est utilisé pour le chargement et la manipulation des données, **re** pour les expressions régulières (pour le prétraitement du texte), **sklearn** pour le topic modeling, **nltk** pour le traitement du langage naturel et **pyLDAvis** pour la visualisation.

In [ ]:
import pandas as pd
import re
import nltk

Nous téléchargeons ensuite les ressources NLTK nécessaires : **wordnet** pour la lemmatisation ainsi que la liste prédéfinie des stopwords.

In [ ]:
# Téléchargement des ressources NLTK pour le prétraitement du texte
nltk.download('wordnet') # package pour la limmatisation
nltk.download('stopwords') # package pour les stop words
nltk.download('punkt') # package pour le tokenisation
nltk.download('averaged_perceptron_tagger') # package pour le POS tagging

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Prétraitement du texte

Nous initalisaons le lemmatizer et la liste des stopwords et nous définissons une fonction **preprocess_text** pour le prétraitement des avis. Cette fonction nettoie le texte, le convertit en minuscules, supprime les stopwords, et applique la lemmatisation.

In [ ]:
# Charger les données
data = pd.read_csv("electronics_sample.csv")
data = data.dropna(subset=['reviewText'])
text_data = data['reviewText'].values
print(text_data[0 : 5])

['Tech support is the worst'
 'Screws were missing from the bracket and beauty dish within a week.\nSpend a little more and get much better.'
 'Trouble connecting and staying connected via bluetooth'
 "I purchased this unit for our RV to replace an older AM/FM/Cassette Tape unit.  I was looking for the CD and Bluetooth capability as well as higher quality sound and this unit certainly delivers all of that.  I found the system for setting up the unit took some getting used to.  I believe this process would be simpler if Alpine had provided 1 or 2 more buttons so there was less button multi-tasking.  It took several attempts to finally get a device recognized and connected on bluetooth.  So far, the receiver has linked to my iPad but refuses to link to my iPod Touch or my cell phone.  It also took several attempts to update the receiver firmware but that was finally achieved.  The update has not solved the bluetooth device linking problem.  I plan to contact Alpine tech support to see if

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialisation du lemmatizer et des stop words
stop_words = set(stopwords.words('english'))

# Initilisation du lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Exemple de part of speech tagging (POS)
def tag_parts_of_speech(sentence):
    # Tokeniser la phrase en mots
    words = nltk.word_tokenize(sentence)

    # Obtenir les étiquettes de partie du discours
    pos_tags = nltk.pos_tag(words)

    return pos_tags

# Exemple d'utilisation
sentence = "The quick brown fox jumps over the lazy dog"
pos_tags = tag_parts_of_speech(sentence)
print(pos_tags)

[('The', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumps', 'VBZ'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN')]


In [ ]:
def preprocess_text(text):
    # Convertir le texte en minuscules et remplacer les caractères non alphabétiques par des espaces
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)

    # Diviser le texte en mots (tokenisation)
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    # Initialiser le lemmatiseur
    #lemmatizer = WordNetLemmatizer()
    filtered_tokens = []
    for token, pos in pos_tags:
      if token not in stop_words:
            # Supprimer les verbes
            if not pos.startswith('VB'):
                # Lemmatizer le token
                lemmatized_token = lemmatizer.lemmatize(token)
                filtered_tokens.append(lemmatized_token)
    return ' '.join(filtered_tokens)

# Application du prétraitement
preprocessed_text = [preprocess_text(text) for text in text_data]

## Chargement des données

Ici, nous chargeons les données à partir d'un fichier CSV, nettoyons les données en supprimant les lignes sans avis, puis appliquons la fonction de prétraitement à chaque avis.

## Vectorisation et modélisation

Avant d'effectuer la modélisation, les mots sont transformés en valeurs numériques (vecteurs) à l'aide de **CountVectorizer**. Celui-ci peut considérer ausi bien des 1-gram que des bigrams, etc. Les mots sont ensuite filtrés à l'aide de **max_df** et **min_df** pour ignorer les mots qui sont trop fréquents ou trop rares. Cela permet à l'algorithme LDA de se concentrer sur les mots significatifs et réduit la dimensionnalité du dataset. Le modèle est entraîné sur la matrice document-terme formée.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorisation du texte avec bigrammes
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english', ngram_range=(1, 2))
dtm = vectorizer.fit_transform(preprocessed_text)

# Création et ajustement du modèle LDA
lda = LatentDirichletAllocation(n_components=5, random_state=0, n_jobs=1)
lda.fit(dtm)

LatentDirichletAllocation(n_components=5, n_jobs=1, random_state=0)

 ## Visualisation des thématiques

 Ce code prépare les données pour **pyLDAvis** et renvoie une visualisation interactive des sujets trouvés dans les avis. Cela permet une exploration visuelle et intuitive des thématiques et de leurs mots clés associés.

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
# Préparation des données pour pyLDAvis
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.lda_model.prepare(lda, dtm, vectorizer, sort_topics=False)

# Affichage de la visualisation
pyLDAvis.display(lda_display)